<a href="https://colab.research.google.com/github/Abonia1/Deep-learning/blob/main/Fine_tune_a_pre_trained_model_for_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Activate GPU and Install Dependencies

In [ ]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

True

In [ ]:
# Install required libraries
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 7.3 MB/s 
     |████████████████████████████████| 4.4 MB 63.4 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 140 kB 72.2 MB/s 
     |████████████████████████████████| 212 kB 72.1 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
     |████████████████████████████████| 596 kB 69.4 MB/s 
     |████████████████████████████████| 127 kB 73.6 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 
     |████████████████████████████████| 144 kB 72.9 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 271 kB 5.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing 

#2. Preprocess data

In [ ]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Create a smaller training dataset for faster training times
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])
print(small_train_dataset[0])
print(small_test_dataset[0])

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}
{'text': "<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<br /><br />Very quickly, 

In [ ]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f9dc3a68d40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Training the model

In [ ]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [ ]:
# Define the evaluation metrics 
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [13]:
# Log in to your Hugging Face account 
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [14]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples into local empty directory.


In [15]:
# Train the model
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 376


Step,Training Loss


Saving model checkpoint to finetuning-sentiment-model-3000-samples/checkpoint-188
Configuration saved in finetuning-sentiment-model-3000-samples/checkpoint-188/config.json
Model weights saved in finetuning-sentiment-model-3000-samples/checkpoint-188/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-3000-samples/checkpoint-188/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-3000-samples/checkpoint-188/special_tokens_map.json
tokenizer config file saved in finetuning-sentiment-model-3000-samples/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-3000-samples/special_tokens_map.json
Saving model checkpoint to finetuning-sentiment-model-3000-samples/checkpoint-376
Configuration saved in finetuning-sentiment-model-3000-samples/checkpoint-376/config.json
Model weights saved in finetuning-sentiment-model-3000-samples/checkpoint-376/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-3000

TrainOutput(global_step=376, training_loss=0.30971005622376785, metrics={'train_runtime': 291.379, 'train_samples_per_second': 20.592, 'train_steps_per_second': 1.29, 'total_flos': 783875831546880.0, 'train_loss': 0.30971005622376785, 'epoch': 2.0})

In [18]:
# Compute the evaluation metrics
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.8766666666666667,
 'eval_f1': 0.877076411960133,
 'eval_loss': 0.299103707075119,
 'eval_runtime': 5.3455,
 'eval_samples_per_second': 56.122,
 'eval_steps_per_second': 3.554}

# 4. Analyzing new data with the model

In [19]:
# Upload the model to the Hub
trainer.push_to_hub()

Saving model checkpoint to finetuning-sentiment-model-3000-samples
Configuration saved in finetuning-sentiment-model-3000-samples/config.json
Model weights saved in finetuning-sentiment-model-3000-samples/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-3000-samples/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-3000-samples/special_tokens_map.json


Upload file runs/Jun29_15-12-14_9f1896b5414e/events.out.tfevents.1656515847.9f1896b5414e.74.2: 100%|##########…

To https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples
   20bdbc2..6c78e56  main -> main



'https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/commit/6c78e5636149b7510d4e2d296f284e753edaab19'

In [20]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="Abonia/finetuning-sentiment-model-3000-samples")

sentiment_model(["I like this movie", "This movie is bad!"])

Downloading:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmps3_vq94b


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

storing https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/36e38352ab1599093e3091df43d2df0a510e504ba4c5e8eeb9e87e80af59f570.424754d7ecb51ff9f4c71669d9f2f2e360304281b49e698fe9eb267a501bf8f0
creating metadata file for /root/.cache/huggingface/transformers/36e38352ab1599093e3091df43d2df0a510e504ba4c5e8eeb9e87e80af59f570.424754d7ecb51ff9f4c71669d9f2f2e360304281b49e698fe9eb267a501bf8f0
loading configuration file https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/36e38352ab1599093e3091df43d2df0a510e504ba4c5e8eeb9e87e80af59f570.424754d7ecb51ff9f4c71669d9f2f2e360304281b49e698fe9eb267a501bf8f0
Model config DistilBertConfig {
  "_name_or_path": "Abonia/finetuning-sentiment-model-3000-samples",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

storing https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/38f7b6b6338eb6b023d4ddcc7216a5e0813b067c2c9f3b0353c647235f0b744f.848c414913cfee271695b8761d3e947fb18a724fbad549de63228b20e5f2d615
creating metadata file for /root/.cache/huggingface/transformers/38f7b6b6338eb6b023d4ddcc7216a5e0813b067c2c9f3b0353c647235f0b744f.848c414913cfee271695b8761d3e947fb18a724fbad549de63228b20e5f2d615
https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpj78rkgos


Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

storing https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/7fed526d47f829ebf5819c211b632931c92a57cc4df738115fa67c61f416d42b.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
creating metadata file for /root/.cache/huggingface/transformers/7fed526d47f829ebf5819c211b632931c92a57cc4df738115fa67c61f416d42b.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
loading file https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/4e707215af021be9f04631d4b248b3ea2d19fe11a8434c230f7c0ef39e808993.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/Abonia/finetuning-sentiment-model-3000-samples/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/38f7b6b6338eb6b023d4ddcc7216a5e0813b067c2c9f3b0353c647235f0

[{'label': 'LABEL_1', 'score': 0.9384443759918213},
 {'label': 'LABEL_0', 'score': 0.916633665561676}]